# Spin pumping program
**`author`**: `Kuan-Chia Chiu`  
**`update`**: `2018-12-11`  
**`package installed`**: `pyvisa`, `serial`, `ipywidgets`

In [1]:
import visa
import time
import serial
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
rm = visa.ResourceManager()
res = rm.list_resources()
vna_dict =[]
for x in range(0, len(res)):
    vna_dict.append(res[x])
    print("{}. ".format(x) + res[x] + ", \n")
print("\n")

0. ASRL4::INSTR, 

1. ASRL5::INSTR, 





In [2]:
print("Choose address of intrsuments:")
VNA_address =widgets.ToggleButtons(
    options=vna_dict,
    description="VNA",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=[],
#     icons=['check'] * 3
)

VNA_address

Choose address of intrsuments:


ToggleButtons(button_style='success', description='VNA', options=('ASRL4::INSTR', 'ASRL5::INSTR'), value='ASRL…

In [3]:
KEPCO_address = widgets.ToggleButtons(
    options=vna_dict,
    description="Power Sup.",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=[],
#     icons=['check'] * 3
)
KEPCO_address

ToggleButtons(button_style='success', description='Power Sup.', options=('ASRL4::INSTR', 'ASRL5::INSTR'), valu…

## Reading Magnetic Field

In [ ]:
ser = serial.Serial('COM3', 57600,bytesize=7, parity=serial.PARITY_ODD, stopbits=1)  # open serial port

print(ser.port)         # check which port was really used
ser.write(b'*IDN?\n')     # write a string
id_name = ser.readline()
#ser.flush()
print(id_name.decode('utf8'))
ser.write(b'RDGFIELD?\n')
Field = ser.readline()
#ser.flush()
print(Field.decode('utf8'))

ser.close()             # close port

## KEPCO Control

In [ ]:
KEPCO.timeout=10000
KEPCO.write("FUNC:MODE CURR")
KEPCO.write("volt  20.000;curr {:.6f}".format(START_CURR))
KEPCO.write("OUTPUT ON")
for i in range(0,10):
    print(i)
    time.sleep(1)
KEPCO.write("OUTPUT OFF")
KEPCO.write("volt 0.000;curr 0.000")


result = KEPCO.read()
print(result)

## VNA Control

In [ ]:
vna.timeout=10000
vna.write("SYST:PRESET")
vna.write("CALC:PAR:DEF 'My_S21', S21")
vna.write("DISPlay:WINDow1:STATe ON")
vna.write("DISPlay:WINDow1:TRACe1:FEED 'My_S21'")
vna.write("INITiate1:CONTinuous OFF;*OPC?")
opc = vna.read() # wait unitl complete

vna.write("SENSe1:SWEep:TRIGger:POINt OFF")
vna.write("SENSe1:SWEep:POINts 201")
vna.write("SENSe1:SWEep:TIME 200")
vna.write("SENSe1:FREQuency:STARt 5000000000")
vna.write("SENSe1:FREQuency:STOP 5000000000")



#### Here start to give a current
sweep_current = 30
sweep_pts = 201
KEPCO = rm.open_resource(KEPCO_address.value)

KEPCO.timeout=10000
KEPCO.write("FUNC:MODE CURR")
KEPCO.write("volt 0.000;curr 0.000")
KEPCO.write("OUTPUT ON")
Field_data = []
Current_data = []
#### Start to run VNA and Power Supply
vna.write("INITiate1;")
for i in range(sweep_pts):
    KEPCO.write("volt  20.000;curr {:.4f}".format(i*sweep_current/(sweep_pts-1)))
    time.sleep(1)
    Current = str(i*sweep_current/(sweep_pts-1))
    Current_data.append(Current)
    ser.write(b'RDGFIELD?\n')
    Field = ser.readline()
    Field_data.append(Field.decode('utf8'))
    time.sleep(0.1)

KEPCO.write("OUTPUT OFF")
KEPCO.write("volt 0.000;curr 0.000")
####

opc = vna.read() # wait until conplete
 
#Must select the measurement before we can read the data
vna.write("CALCulate1:PARameter:SELect 'My_S11'")
 
#Read the measurement data into the "result" string variable
vna.write("FORMat ASCII")
vna.write("CALCulate1:DATA? FDATA")
result = vna.read()
print(result)